# `Formation Consortium Ariane: Lyon 8-9 novembre 2023`
##### [Motasem Alrahabi](https://obtic.sorbonne-universite.fr/alrahabi/index.html), [ObTIC](https://obtic.sorbonne-universite.fr/) - [Sorbonne Université](https://www.sorbonne-universite.fr/)
### `Lien Github:` https://github.com/Consortium-ARIANE/Formation-Lyon-8-9-novembre-2023/tree/main/carto

### `Lien Colab:` https://colab.research.google.com/drive/1Es7sDrnOnhLtjo-7OBraEqRkW32OZq7x?usp=sharing  

---

## `Introduction`

*   Le traitement automatique des langues ([TAL](https://fr.wikipedia.org/wiki/Traitement_automatique_du_langage_naturel)) est un domaine qui permet aux ordinateurs de comprendre, de générer et de manipuler le langage humain.

*   L'objectif du TAL est d'analyser les données langagières (écrites ou parlées), afin d'effectuer un certain nombre de tâches comme la classification, la génération, la réponse à des questions, le résumé, etc.

*   Généralement, on applique une série de prétraitements aux données, afin de réaliser les différentes tâches: tokenisation (diviser le texte en "tokens"), nettoyage du texte (supprimer les caractères spéciaux, les symboles...), suppression des mots courants ou stopwords, conversion en minuscules, normalisation (lemmatisation ou désinence), encodage ou vectorisation (convertir les tokens en une représentation numérique adaptée à l'entrée du modèle), détection de la langue, etc.

*   Les modèles pré-entrainés: modèles de neuronnes pré-entrainés sur un grand nombre de données.

*   Les Transformers sont une classe d'architectures de réseaux de neurones artificiels. Ils ont la capacité de gérer des séquences de données, comme du texte, de manière plus efficace que les architectures précédentes.

*   [Hugging Face](https://huggingface.co/) est une librairie qui fournit des API et des outils pour utiliser les modèles pré-entrainés.

*   Hugging Face Pipeline simplifie l'utilisation de modèles pour des tâches courantes: classification, résumé, traduction, QR, etc.

*   Dans cette formation on utilisera uniquement des modèles existants (une suite logique serait de former de nouveaux modèles: entrainement, évaluation, etc.).

*   Pour la formation, on va utiliser google collab qui offre l'avantage d'utiliser des GPU gratuitement.



---




# `Partie 1: utiliser les transformers pour réaliser différentes tâches en TAL`:

### `1-1) Importer quelques librairies nécessaires`:

In [ ]:
!pip install --upgrade -q transformers
!pip install -q sentencepiece
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForCausalLM, AutoModelWithLMHead
import pandas as pd

### `1-2) Etiquetage morpho-syntaxique (parties du discours, POS)`:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gilf/french-camembert-postag-model")
model = AutoModelForTokenClassification.from_pretrained("gilf/french-camembert-postag-model")
categories = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)
texte = "Je suis étudiant et je vis à Lyon"
categories(texte)
#Liste des catégories: https://huggingface.co/gilf/french-camembert-postag-model

### `1-3) Reconnaissance d'entités nommées (NER)`:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple", grouped_entities=True)
texte = "Carla travaille à l'ONU à Paris"
pd.DataFrame(ner(texte))
# https://huggingface.co/Davlan/bert-base-multilingual-cased-ner-hrl

### `1-4) Classification de textes`:

In [ ]:
classifieur = pipeline("zero-shot-classification")
texte = "Cette formation porte sur l'utilisation de la bibliothèque Hugging Face"
thematique = ["technologie", "éducation", "société", "formation"]
classifieur(texte, thematique)
#https://huggingface.co/tasks/zero-shot-classification

### `1-5) Analyse des sentiments et des émotions`:

In [ ]:
# utilisation du modèle par défaut
classifieur = pipeline("sentiment-analysis")
classifieur("Cette formation est très cool. Je la recommande à mes amis")

In [ ]:
# On peut aussi personnaliser le modèle:
classifieur = pipeline(model="citizenlab/twitter-xlm-roberta-base-sentiment-finetunned")
classifieur("Cette formation est inutile. Je ne la recommande pas à mes amis")
# https://huggingface.co/citizenlab/twitter-xlm-roberta-base-sentiment-finetunned

In [ ]:
# Analyse des émotions sur l'anglais (Ekman classif.):
classifier = pipeline("text-classification", model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
classifier("I love using transformers. The best part is wide range of support and its easy to use", )
# https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion

In [ ]:
# un autre exemple proche, et qui donne un seul résultat:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")
text = "i feel as if i haven't blogged in ages or at least truly blogged. I am doing an update. Cute."# Output: 'joy'
#text = "i have a feeling i kinda lost my best friend" # Output: 'sadness'
tokenizer.decode(model.generate(tokenizer.encode(text + '</s', return_tensors='pt'), max_length=2)[0])
# https://huggingface.co/mrm8488/t5-base-finetuned-emotion

In [ ]:
# Analyse des émotions sur l'anglais (goEmotions classif.):
classifier = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
classifier("I am not having a great day")
# https://huggingface.co/SamLowe/roberta-base-go_emotions

### `1-6) Génération de textes`:

In [ ]:
generateur = pipeline("text-generation", model = "bigscience/bloom-560m")
prompt = "Cette formation va vous permettre de"
generateur(prompt, max_length = 40)
# https://huggingface.co/bigscience/bloom-560m

### `1-7) Question-Réponse (QA)`:

In [ ]:
question_reponse = pipeline("question-answering", model="cmarkea/distilcamembert-base-qa", tokenizer="cmarkea/distilcamembert-base-qa")
texte = """David Fincher, né le 28 août 1962 à Denver (Colorado),
    est un réalisateur et producteur américain. Il est principalement
    connu pour avoir réalisé les films Seven, Fight Club, L'Étrange
    Histoire de Benjamin Button, The Social Network et Gone Girl qui
    lui ont valu diverses récompenses et nominations aux Oscars du
    cinéma ou aux Golden Globes. """
question_reponse(context = texte, question = "Quel est le métier de David Fincher ?")
# https://huggingface.co/cmarkea/distilcamembert-base-qa

### `1-8) Résumé automatique`:

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, SummarizationPipeline
model = T5ForConditionalGeneration.from_pretrained("plguillou/t5-base-fr-sum-cnndm")
tokenizer = T5Tokenizer.from_pretrained("plguillou/t5-base-fr-sum-cnndm")
resumeur = SummarizationPipeline(model=model, tokenizer=tokenizer)

texte = """David Fincher, né le 28 août 1962 à Denver (Colorado),
    est un réalisateur et producteur américain. Il est principalement
    connu pour avoir réalisé les films Seven, Fight Club, L'Étrange
    Histoire de Benjamin Button, The Social Network et Gone Girl qui
    lui ont valu diverses récompenses et nominations aux Oscars du
    cinéma ou aux Golden Globes. Réputé pour son perfectionnisme, il
    peut tourner un très grand nombre de prises de ses plans et
    séquences afin d'obtenir le rendu visuel qu'il désire. Il a
    également développé et produit les séries télévisées House of
    Cards et Mindhunter, diffusées sur Netflix."""

resumeur(texte, min_length=20, max_length=100, clean_up_tokenization_spaces=True)
# https://huggingface.co/plguillou/t5-base-fr-sum-cnndm

### `1-9) Traduction`:

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
traducteur = pipeline("translation_en_to_fr", model=model, tokenizer=tokenizer)
traducteur("My name is Bob and I work in Japan")
# https://huggingface.co/t5-small

### `1-10) Conversation`:

In [ ]:
from transformers import Conversation
#chatbot = pipeline("conversational")
chatbot = pipeline(model="microsoft/DialoGPT-medium")
conversation = Conversation("I would like to watch a movie today, any suggestion?")
conversation = chatbot(conversation)
print()
print("Q - I would like to watch a movie today, any suggestion?")
print("A - ", conversation.generated_responses[-1])

conversation.add_user_input("Is it an action movie?")
conversation = chatbot(conversation)
print("Q - Is it an action movie?")
print("A - ", conversation.generated_responses[-1])
# https://huggingface.co/microsoft/DialoGPT-medium?text=Hi.

### `1-11) Comparaison`:

In [ ]:
! pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")
sentences = ["Un avion est en train de décoller.",
          "Un homme joue d'une grande flûte.",
          "Un homme étale du fromage râpé sur une pizza.",
          "Une personne jette un chat au plafond.",
          "Une personne est en train de plier un morceau de papier.",
          ]
model.encode(sentences)
# https://huggingface.co/dangvantuan/sentence-camembert-large



---

# `Partie 2: Cartographie émotionnelle des noms de lieux`

#`Introduction`:
La cartographie émotionnelle des toponymes est une méthode qui consiste à associer des émotions à des lieux géographiques spécifiques.

Cette méthode peut être utilisée pour explorer la manière dont les lieux sont perçus et ressentis par les gens qui y vivent ou qui les visitent.

Exemples: [Victorian London](https://www.historypin.org/en/victorian-london/geo/51.5128,-0.116085,12/bounds/51.423936,-0.222,51.601491,-0.01017/paging/1) project, [Mapping Arabia](https://www.google.com/maps/d/u/0/edit?hl=fr&mid=1M--Wq2CJcCPjIccrm5h8LYoTANTTH4cc&ll=48.84129625249132%2C2.356532365466477&z=13), etc.


Pour générer une carte avec les noms de lieu qui se trouvent dans un texte, on peut suivre les étpaes suivantes:

1- **`Extraire les noms de lieu du texte`** en utilisant des techniques de NLP pour détecter les entités géographiques dans le texte: noms de villes, de pays, de rivières, de montagnes, etc.

Difficultés: Couverture: les EN sont une liste ouverte...(mots inconnus, variantes graphiques et historiques, abréviations...) ; Granularité et délimitation: imbrication, portée...(jardin du Luxembourg) ; Homonymie: Paris (France) vs Paris Hilton ; Orange (ville) vs Orange (fruit) ; Métonymique: Charles de Gaulle (personne, aéroport, porte-avion, etc.)...

2- **`Géocodage`** : envoyer les noms de lieu à un service de géocodage pour convertir les noms de lieu en coordonnées géographiques (latitude et longitude). On peut le faire en utilisant une bibliothèque de géocodage ou en faisant des requêtes à un service de géocodage en ligne.

Difficultés: la désambiguïsation des noms de lieu qui se trouvent en doublons. Ex: Paris (France) vs Paris (Etas-Unis) ; Tunis (pays) ou Tunis (capitale). On peut aussi assurer la liaison de données (linking data) qui permet de relier à un référentiel unique une ou plusieurs entités (Ex. 1 av. de l'Exemple, Paris -> 1 avenue de l'Exemple, 75005 Paris). On utilise des bases comme Wikidata.

3- **`Affichage sur une carte`** : afficher les coordonnées géographiques des lieux sur une carte en utilisant une bibliothèque de cartographie comme Leaflet, Google Maps, Mapbox, etc. On pourra créer des marqueurs sur la carte pour chaque lieu et y afficher le nom du lieu en tant qu'infobulle ou étiquette.



---



**Ce code effectue les opérations suivantes :**
1. Le texte en entrée est segmenté, phrase par phrase (avec Spacy).
2. Chaque phrase est parcourue pour toutes les entités nommées LOC (avec Spacy). Il est possible de remplacer le modèle utilisé par défaut avec un autre. Une liste de termes indésirables peut également être personnalisée par l'utilisateur.
3. Pour chaque entité trouvée, le script recherche les coordonnées géographiques. Il utilise la base de données Geonamescache comme source principale. En cas d'absence de résultat, il se connecte au serveur Nominatim afin de géocoder les lieux (les villes dans ce script).
4. Si au moins une entité nommée est trouvée dans une phrase, une nouvelle fonction analyse les sentiments positifs, négatifs et neutres de la phrase entière (en utilisant Bert-base-multilingual-uncased-sentiment - qui peut d'ailleurs être remplacé par un autre modèle).
5. A l'aide de la bibliothèque Python Folium (basée sur LeafLet), le script affiche sur une carte les entités nommées trouvées. La couleur de chaque icône est calculée en fonction du sentiment moyen de cette entité dans l'ensemble du texte : vert pour positif, rouge pour négatif et gris pour neutre. La taille de l'icône est proportionnelle au nombre d'occurrences dans l'ensemble du texte. Pour chaque icône, une fenêtre contextuelle affiche le nom du lieu, le numéro d'occurrence, le nombre de sentiments positifs, négatifs et neutres dans le texte saisi.
6. Le script enregistre chaque entité dans un fichier csv avec les informations suivantes : fichier, phrase, sentiment, latitude et longueur.



---



## `0) Quelques exercices avant de commencer`:

Pour rechercher les coordonnées géographiques d'un nom de lieu:

In [ ]:
! pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='my-app', timeout=10)
location = geolocator.geocode("Lyon")
print("Coordonnées de Lyon : Latitude:", location.latitude, " / Longitude:", location.longitude)

Pour effectuer une recherche inverse (reverse geocoding) et récupérer le nom de lieu à partir de ses coordonnées géographiques:

In [ ]:
latitude = 45.75
longitude = 4.85
location = geolocator.reverse((latitude, longitude), exactly_one=True)
if location is not None:
    print(f"Nom de lieu le plus proche : {location.address}")
else:
    print("Aucun lieu trouvé pour les coordonnées spécifiées.")

Pour afficher la ville de Lyon sur une carte :


In [ ]:
! pip install folium
import folium
lyon_map = folium.Map(location=[45.75, 4.85])
folium.Marker(location=[45.75, 4.85], popup="Formation Consortium ARIANE - Lyon").add_to(lyon_map)
lyon_map



---

## Analyser des sentiments autour des noms de lieu


## `1) Télécharger les bibliothèques nécessaires`:

In [11]:
! pip install -q spacy geopy folium transformers
! pip install -q --upgrade tensorflow
! pip install -q geonamescache

import os
import csv
import spacy
import folium
import requests
import geonamescache

from folium import plugins
from spacy import displacy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from google.colab import files, output

## `2) Charger le modèle de reconnaissance d'entités nommées`:

In [ ]:
! python -m spacy download fr_core_news_md
nlp = spacy.load('fr_core_news_md') # l'utilisateur peut ici charger le modèle qu'il souhaite.

## `3) Charger le modèle d'analyse de sentiments`:

In [13]:
from transformers import pipeline
sentiment_analysis_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", framework="pt")
#https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

## `4) Charger une liste de terms d'entités nommées à exclure`:


In [14]:
blacklist = ["XVIIIe", "XVIe"] # juste pour l'exemple

## `5) Segmenter le texte en phrases`:

In [15]:
def segmenter_en_phrases(text):
  all_sentences = []
  text = text.replace('’', "'") # normalisation
  doc = nlp(text)
  sentences = [sent.text for sent in doc.sents]
  all_sentences.extend(sentences)
  #print(all_sentences) # toutes les phrases du corpus dans une liste
  return all_sentences

## `6) Traiter chaque phrase: identifier les entités, les coordonnées et les sentiments`.

In [16]:
def analyze_sentence(sentence):
  doc = nlp(sentence)
  analyzed_sentence = []
  entities = []
  latitude = None
  longitude = None
  sentiment_score_sentence = 0
  sentiment_label_sentence = ""

  for ent in doc.ents:
      if ent.label_ == "LOC":  # je ne prends pas les autres types d'entités (PER, ORG...)
        if ent.text.lower() not in blacklist:
            entities.append((ent.text, ent.start, ent.end))

            # Charger la bdd geonames:
            gc = geonamescache.GeonamesCache()
            matching_cities = gc.get_cities_by_name(ent.text)

            if matching_cities: #seulement les villes dans ce script
                first_matching_city = list(matching_cities[0].values())[0]
                latitude = first_matching_city['latitude']
                longitude = first_matching_city['longitude']
            else:
                # Si la ville n'st pas trouvée dans geonamescache, utiliser Nominatim comme une deuxième option:
                geolocator = Nominatim(user_agent='my-app', timeout=10)
                try:
                    location = geolocator.geocode(ent.text)
                    if location:
                      latitude = location.latitude
                      longitude = location.longitude
                    else:
                      raise GeocoderTimedOut("Geocoding timed out for location using Nominatim: " + ent.text)
                except GeocoderTimedOut as e:
                  print(f"Geocoding timed out for location using Nominatim: {ent.text}")
                  latitude = None
                  longitude = None
                  location = None

            if entities and latitude and longitude:
              sentiment_score_sentence = sentiment_analysis_model(sentence)[0]["score"]
              sentiment_label_sentence = sentiment_analysis_model(sentence)[0]['label']
            result = {
                "entity": ent.text,
                "sentiment_label_sentence": sentiment_label_sentence,
                "sentiment_score_sentence": sentiment_score_sentence,
                "latitude": latitude,
                "longitude": longitude
            }
            analyzed_sentence.append(result)

  return analyzed_sentence

## `7) Charger le corpus: choisir l'une de ces 2 options`:

### 7-1) Le texte input est fourni dans le code


In [ ]:
text = """J'adore Paris pour son atmosphère romantique et ses délicieuses pâtisseries.
New York m'impressionne avec ses gratte-ciel imposants, mais je trouve la ville un peu trop bruyante à mon goût.
La campagne française est d'une beauté paisible qui apaise l'âme.
Je suis neutre à l'égard de Los Angeles, la ville a ses avantages, mais le trafic peut être vraiment agaçant.
Venise m'a profondément ému avec ses canaux pittoresques et son histoire fascinante.
Londres a une ambiance unique qui me rend nostalgique de mes voyages passés.
Je n'aime pas du tout les hivers rigoureux de Montréal, mais l'été est absolument charmant.
La plage de Bali est un véritable paradis, et j'adore passer du temps là-bas.
Les déserts du Sahara sont à couper le souffle, mais la chaleur peut être accablante.
Tokyo m'a laissé une impression mitigée, certains quartiers sont incroyablement modernes, tandis que d'autres conservent une atmosphère traditionnelle qui est fascinante."""

all_sentences = segmenter_en_phrases(text)

### 7-2) Le texte est un fichier sur PC

In [17]:
uploaded = files.upload()
file_name = next(iter(uploaded))
if os.path.isfile(file_name) and file_name.endswith(".txt"):
  with open(file_name, "r", encoding="utf-8") as file:
    text = file.read()
    all_sentences = segmenter_en_phrases(text)
else:
  print(f"Le fichier {file_name} n'est pas au format .txt et ne sera pas traité.")

Saving Stefan Zweig MAGELLAN.txt to Stefan Zweig MAGELLAN (1).txt


## `8) Parcourir et agréger les résultats`:

In [ ]:
# Spécifier ici une limite de phrases à traiter (pour réduire le temps de traitement):
limite = 100

# Enregistrr les entités dans un CSV file:
analyzed_sentences = []

with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter="\t")
    writer.writerow(['sentence', 'entity', 'label', 'score', 'latitude', 'longitude'])
    counter = 0
    # Analyze each sentence
    for sentence in all_sentences:
      if counter>=limite: # le nombre maximum de phrases à analyser
        break
      analyzed_sentence = analyze_sentence(sentence)
      for result in analyzed_sentence:
        entity = result["entity"]
        sentiment_label = result["sentiment_label_sentence"]
        sentiment_score = result["sentiment_score_sentence"]
        latitude = result["latitude"]
        longitude = result["longitude"]

        # check if lat & long are None
        if latitude is None or longitude is None:
            print(f"Skipping result due to missing location data for entity: {entity}")
            continue  # skip this sentence and move to the next one

        writer.writerow([sentence, entity, sentiment_label, round(sentiment_score, 2), latitude, longitude])
        counter += 1
        print(f"Sentiment de la phrase {counter}: {sentence[:20]}... ; entité:[{entity}] ; polarity: {sentiment_label} ; score: {round(sentiment_score, 2)}")
        analyzed_sentences.append({
            'sentence': sentence,
            'entity': entity,
            'sentiment_label': sentiment_label,
            'sentiment_score': round(sentiment_score, 2),
            'latitude': latitude,
            'longitude': longitude
        })


# Itérer sur les analyzed_sentences et créer un nouveau dico:
entities_dict = {}
for sentence_info in analyzed_sentences:
    entity = sentence_info['entity']
    sentiment_label = sentence_info['sentiment_label']

    # Initialisere les infos de l'entités si ce n'est pas encore fait
    if entity not in entities_dict:
        entities_dict[entity] = {
            'latitude': sentence_info['latitude'],
            'longitude': sentence_info['longitude'],
            'occurrences': 1,
            'positive_labels': 0,
            'negative_labels': 0,
            'neutral_labels': 0,
            'overall_sentiment': sentiment_label
        }
    else:
        entities_dict[entity]['occurrences'] = entities_dict[entity]['occurrences'] + 1

    # màj le calcul des sentiments
    if sentiment_label in ['4 stars', '5 stars']:
        entities_dict[entity]['positive_labels'] += 1
    elif sentiment_label in ['1 star', '2 stars']:
        entities_dict[entity]['negative_labels'] += 1
    elif sentiment_label == '3 stars':
        entities_dict[entity]['neutral_labels'] += 1

    # Update overall sentiment based on counts
    for entity in entities_dict:
      positive_count = entities_dict[entity]['positive_labels']
      negative_count = entities_dict[entity]['negative_labels']
      neutral_count = entities_dict[entity]['neutral_labels']

      if positive_count > negative_count and positive_count > neutral_count:
        entities_dict[entity]['overall_sentiment'] = 'Positive'
      elif negative_count > positive_count and negative_count > neutral_count:
        entities_dict[entity]['overall_sentiment'] = 'Negative'
      elif neutral_count > positive_count and neutral_count > negative_count:
        entities_dict[entity]['overall_sentiment'] = 'Neutral'
      else:
        entities_dict[entity]['overall_sentiment'] = 'Mixed'

#print(entities_dict)


## `9) Visualisation`:

In [ ]:
# Initialiser le map
map_center = [0, 0]
map_zoom = 2.5
map = folium.Map(location=map_center, zoom_start=map_zoom, tiles='CartoDB Positron')

# parcourir les entités
for entity, info in entities_dict.items():
  latitude = info['latitude']
  longitude = info['longitude']
  occurrences = info['occurrences']
  overall_sentiment = info['overall_sentiment']
  positive_labels = info['positive_labels']
  negative_labels = info['negative_labels']
  neutral_labels = info['neutral_labels']

  # Déterminer la couleur de l'entité selon le sentiment général
  if overall_sentiment == 'Positive':
    color = 'green'
  elif overall_sentiment == 'Negative':
    color = 'red'
  elif overall_sentiment == 'Neutral':
    color = 'gray'
  elif overall_sentiment == 'Mixed':
    color = 'blueviolet'
  else:
    color = 'blue'
  #print(overall_sentiment,"  ", color)

  scaling_factor = 3  # Ajuster le size scaling des icones
  size = int(occurrences) * scaling_factor

  # Créer la pop up des icones
  popup =  f"<div style='width: 110px; height: 80px;'>"
  popup += f"<b>Entity:</b> {entity}<br>"
  popup += f"<b>Occurrences:</b> {occurrences}<br>"
  popup += f"<b>Sentiment:</b> {overall_sentiment}<br>"
  popup += f"<b>Positive:</b> {positive_labels}<br>"
  popup += f"<b>Negative:</b> {negative_labels}<br>"
  popup += f"<b>Neutral:</b> {neutral_labels}<br>"
  popup +=  "</div>"

  folium.CircleMarker(
      location=[latitude, longitude],
      radius=size,
      popup=popup,
      color=color,
      fill=True,
      fill_color=color,
      #tooltip=f"Entity: {entity}"
      tooltip=entity
  ).add_to(map)


# Création de la légende de tous les résultats:
total_entities = len(entities_dict)
total_positive = sum(info['positive_labels'] for info in entities_dict.values())
total_negative = sum(info['negative_labels'] for info in entities_dict.values())
total_neutral  = sum(info['neutral_labels'] for info in entities_dict.values())
total_sentiments = total_positive + total_negative + total_neutral
html_text = """
    <div style="position: absolute;
                top: 50%; left: 0; transform: translate(0, -50%);
                z-index: 1000; background-color: white; border: 2px solid #ccc;
                padding: 10px; font-family: 'Trebuchet MS', sans-serif;">
    <h4>Annotation Results:</h4>
    <b>Total Entities: {}<br>
    <b>Total Sentiments: {}<br>
    <ul>
    <li>Positive: {}</li>
    <li>Negative: {}</li>
    <li>Neutral: {}</li>
    </ul>
    </div>
    """.format(total_entities, total_sentiments, total_positive, total_negative, total_neutral)

# Ajouter la légende à la carte
map.get_root().html.add_child(folium.Element(html_text))

map.save("map.html") # enregistrer le fichier sur le disque
map



---


#`Quelques liens utiles`:

[Ariane](https://obtic.huma-num.fr/ariane/): plateforme d'analyse sémantique.

[Obvie](https://obtic.huma-num.fr/obvie/): outil de fouille lexicale.

[Elicom](https://obtic.huma-num.fr/elicom/): outil de fouille des correspondances.

[Tanagra](https://obtic.sorbonne-universite.fr/tanagra/map): outil de cartographie.

[Pandore](https://pandore-toolbox.isir.upmc.fr/): une boîte à outil pour les humanités numériques.
...



---


#`Fin`

<p xmlns:cc="http://creativecommons.org/ns#" >Ce support est distribué selon les termes de la licence Creative Commons <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">CC BY-NC-SA 4.0

<img style="height:11px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"><img style="height:11px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1"><img style="height:11px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/nc.svg?ref=chooser-v1"><img style="height:11px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1"></a></p>
